Name: Aaron Palpallatoc

Section: S

# Regularized Linear Regression Exercise

**In this exercise you will:**
- Extend your linear regression model to produce non-linear regression lines/curve (sometimes referred to as polynomial regression)
- Include regularization to reduce over-fitting (more specifically implement lasso and ridge regression)

## Instructions
* Read each cell and implement the TODOs sequentially. The markdown/text cells also contain instructions which you need to follow to get the whole notebook working.
* Do not change the variable names unless the instructor allows you to.
* Answer all the markdown/text cells with "A: " on them. The answer must strictly consume one line only.
* You are expected to search how to some functions work on the Internet or via the docs. 
* There are commented markdown cells that have crumbs. Do not delete them or separate them from the cell originally directly below it.  
* You may add new cells for "scrap work" as long as the crumbs are not separated from the cell below it.
* The notebooks will undergo a "Restart and Run All" command, so make sure that your code is working properly.
* You are expected to understand the data set loading and processing separately from this class.
* You may not reproduce this notebook or share them to anyone.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import csv
import math

plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

## Generate Data
Generate some sample data points, and use a sine function over `x` for the `y` values (with random noise).

In [ ]:
def generate_sample_data(pts):
    x = np.random.uniform(-1.0,1.0, pts)
    y = np.sin(6 * x) + np.random.randn(pts) * 0.5
    return x, y

In [ ]:
pts = 20
np.random.seed(1)
over_x, over_y = generate_sample_data(pts)

plt.scatter(over_x,over_y)

## Bias-variance trade-off

### Plotting the optimal model

Look at the figure above. The code who generated the point of that graph follows a sine function, so we know exactly what the best looking model for the data above should look like:

In [ ]:
plt.scatter(over_x,over_y)

x_range = np.arange(-1, 1, 0.01)
y_range = np.sin(6 * x_range)
plt.plot(x_range, y_range, 'k--')

The black dotted line is the target function `f(x)` that generated the red dots (after some additional errors)

Take note that in real world scenarios, we are talking about hundreds to millions of features. So, the ability to "look at the data and just draw the curve that feels right" is a luxury we can enjoy now with this small dataset. With real world data, this is not the case, and we can most likely cannot visualize all features altogether in one graph.

### Plotting a plain analytical solution model 
Plotting the analytical solution/closed form/normal equation model 

To do this, use the function `np.polyfit()` which is similar to the closed form equation from the last notebook. We won't do any feature transformation yet, i.e., just degree == 1. Assign the return value to variable `weights`.

In [ ]:
# Write your code here


The function `np.poly1d()` returns a function that will perform the feature transform for us given the weights. Assign the return value to variable `fit_fn`.

In [ ]:
# Write your code here


`y_range` uses `fit_fn` to transform the data prior to getting the score/predictions (no effect for now because it is still degree=1)

In [ ]:
x_range = np.arange(np.min(over_x), np.max(over_x), 0.01)
y_range = fit_fn(x_range)

Then, we finally plot the analytical solution model

In [ ]:
plt.plot(x_range, y_range, 'k')
plt.scatter(over_x, over_y)

It is clear that this line, no matter which way you change the slope to another direction, will still not do a great job. This is because it is simply a line that we are forcing to model a sine functiom. We call this underfitting because we are not fitting the model to the data well. We also say that this model has a high bias, because our estimate is far off the actual data. Look at the chart below at the far left:

<img width="700px" src="https://imgur.com/mJLDfBs.png" />

One way to remedy this is to increase the order or degree of our hypothesis function through feature transformation. With polynomial feature transformation, we can generate more complex functions, and get a better fit of our dataset.

Let's try to do the same thing, and increase the polynomial degree complexity of our model. 


We normally do this by adding features based on our original features. The `np.polyfit()` function has a way to increase the polynomial degree on its own.

### Plotting a more complex analytical solution model 

This time, let's perform feature transform in our data.

In [ ]:
x_range = np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01)

Perform feature transform to our data by using the function `np.polyfit()` and setting the degree to `3`. Assign the return value to variable `weights`. Then, assign the feature transform function to variable `fit_fn`.

In [ ]:
# Write your code here


Plot the resulting model.

In [ ]:
y_range = fit_fn(x_range)
plt.plot(x_range, y_range, 'k')
plt.scatter(over_x, over_y)

Perform feature transform to our data by using the function `np.polyfit()` and setting the degree to `5`. Assign the return value to variable `weights`. Then, assign the feature transform function to variable `fit_fn`.

In [ ]:
# Write your code here


Plot the resulting model.

In [ ]:
y_range = fit_fn(x_range)
plt.plot(x_range, y_range, 'k')
plt.scatter(over_x, over_y)

Perform feature transform to our data by using the function `np.polyfit()` and setting the degree to `8`. Assign the return value to variable `weights`. Then, assign the feature transform function to variable `fit_fn`.

In [ ]:
# Write your code here


Plot the resulting model.

In [ ]:
y_range = fit_fn(x_range)
plt.plot(x_range, y_range, 'k')
plt.scatter(over_x, over_y)

Perform feature transform to our data by using the function `np.polyfit()` and setting the degree to `12`. Assign the return value to variable `weights`. Then, assign the feature transform function to variable `fit_fn`.

In [ ]:
# Write your code here


Plot the resulting model.

In [ ]:
y_range = fit_fn(x_range)
plt.plot(x_range, y_range, 'k')
plt.scatter(over_x, over_y)

Look at the last generated model (polynomial degree of 12). It definitely got a lot of data correct, but we know that this is not the sine wave function where the data actually came from and what we want to model.

You may also notice that it dipped/peaked into some points that are unneccessary. It has freedom to do this as long as the data it wants to target are met.

We call this overfitting, because while it will work well with this particular set of data, it might do badly with other data points. Overfitting gives us a low training error (good!), but a high test error (bad!). 

We're going to generate more sample (100) data points and treat them as our test data. Take note that we are taking this from our original target function `f(x)`.

In [ ]:
pts = 100
test_x, test_y = generate_sample_data(pts)

We will plot the test data as blue `+`s, and reuse the red dots for our train data. Then let us see how our model was able to capture our new data from the same distribution.

In [ ]:
plt.scatter(over_x, over_y)
plt.scatter(test_x, test_y, marker='+')

x_range = np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01)
y_range = fit_fn(x_range)
plt.plot(x_range, y_range, 'k')

You can now see that our training data (red dots) are covered correctly, but the test data (blue +) from the same distribution as our training data are mostly missed. So, yes, this complex model captured our training data well. But since our data is also inherently noisy, this causes the model to fit the noise as well.

Observe what will happen if we try different values for the degree `order`:

In [ ]:
orders = np.arange(1, 18, 2)
orders

Complete the code below to display the plot of the resulting model for each order in variable `orders`.

In [ ]:
weights = {}
plt.figure(figsize=(10, 10))
for i in range(len(orders)):
    order = orders[i]

    # Write your code here
    weight = None
    fit_fn = None
    
    plt.subplot(3, 3, i + 1)
    plt.scatter(over_x, over_y)

    X_range = np.arange(np.min(over_x), np.max(over_x), 0.01)
    y_range = fit_fn(X_range)
    plt.plot(X_range, y_range, 'k')
    plt.title('Order = ' + str(orders[i]))
    
    # This will get the weights of the generated model
    weights[order] = weight

**Sanity Check:** You should see 9 figures in a 3x3 grid. Each grid shows the hypothesis model in increasing complexity.

**Question #1:** Does the model with the order set to 15 have high or low bias? Does it exhibit low or high variance?

<!--crumb;qna;Q1-->

A: 

Print the weights of a model that is underfit.

In [ ]:
# Write your code here


And, print the weights of a model that is overfit.

In [ ]:
# Write your code here


And, print the weights of a model that fits the data well.

In [ ]:
# Write your code here


**Question #2:** What are the notable differences in terms of the values of the weights of each of the models (aside from the number of weights)?

<!--crumb;qna;Q2-->

A: 

Here are the weights of each of the models:

In [ ]:
for i in orders:
    length = weights[i].shape[0]
    curweights = weights[i]
    print('order =', i, end='\t')
    for weight in curweights:
        print(str(weight),end='\t')
    print('\n')

**Question #3:** Do we expect a low or high train loss for an unregularized model with a polynomial degree of 17?

<!--crumb;qna;Q3-->

A: 

**Question #4:** Do we expect a low or high test loss for an unregularized model with a polynomial degree of 17?

<!--crumb;qna;Q4-->

A: 

### Checking the performance of the model

We know that unregularized models can overfit with a high polynomial degree of complexity, but we have yet to see its actual performance in RMSE. 

Before we compute that, let us import `UnregularizedLinearRegressor` class.

Import the `UnregularizedLinearRegressor` class.

In [ ]:
from regularized_linear_regression import UnregularizedLinearRegressor

Instantiate a `UnregularizedLinearRegressor` object. Set the degree to 1. Assign the object to variable `regressor`.

In [ ]:
# Write your code here


Open `regularized_linear_regression.py` file and complete the `compute_weights()` function in the `UnregularizedLinearRegressor` class. This function computes the weights using the `np.polyfit()` function.

This function has 2 parameters - `X` and `y`, where `X` is an `np.ndarray` of shape (N,) containing the training data and `y` is an `np.ndarray` of shape (N,) containing the ground truth values.

Implement the `compute_weights()` function in the `UnregularizedLinearRegressor` class. Inline comments should help you in completing the contents of the function.

Compute the weights by calling the function `compute_weights()` and assign the return value to variable `W`. 

In [ ]:
# Write your code here


Open `regularized_linear_regression.py` file and complete the `predict()` function in the `UnregularizedLinearRegressor` class. This function predicts values for the test data.

Implement the `predict()` function in the `UnregularizedLinearRegressor` class. Inline comments should help you in completing the contents of the function.

Then, make predictions on the training data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Display the predictions.

In [ ]:
y_predicted

Then, check how well it performed by computing for the MSE between the predicted output vs the actual output

Import the `compute_RMSE()` function.

In [ ]:
from regularized_linear_regression import compute_RMSE

You may use your code for this function from the previous notebook on linear regression. Let's compute the RMSE between the predicted value and the actual value. Assign the return value to variable `train_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('UnregularizedLinearRegressor (order=1) train error: ' + '{:.4f}'.format(train_rmse))

Now, get the predictions and compute for the RMSE on our test data. Assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('UnregularizedLinearRegressor (order=1) test error: ' + '{:.4f}'.format(test_rmse))

Now, let's increase the degree.

Instantiate a `UnregularizedLinearRegressor` object. Set the degree to 30. Assign the object to variable `regressor`.

In [ ]:
# Write your code here


Train the model.

In [ ]:
# Write your code here


Make predictions on the training data and assign the return value to variable `y_train_predicted`. 

Make predictions on the test data and assign the return value to variable `y_test_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the train data and assign the return value to variable `train_rmse`. 

Compute the RMSE on the test data and assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('UnregularizedLinearRegressor (order=30) train error: ' + '{:.4f}'.format(train_rmse))
print('UnregularizedLinearRegressor (order=30) test error: ' + '{:.4f}'.format(test_rmse))

**Question #5:** Is the model above high bias or low bias?

<!--crumb;qna;Q5-->

A: 

**Question #6:** Is the model above high variance or low variance?

<!--crumb;qna;Q6-->

A: 

Visualize the `UnregularizedLinearRegressor` model. The code below is nothing new, we just visualize our `UnregularizedLinearRegressor`. It should look similar to the previous models.

In [ ]:
plt.scatter(over_x, over_y)
plt.scatter(test_x,test_y, marker='+')

x_range = np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01)
y_range = regressor.predict(x_range)
plt.plot(x_range, y_range, 'k')

plt.title('UnregularizedLinearRegressor (order=30) model')

We'll use `UnregularizedLinearRegressor` again later in the notebook.

**Note:** Before we get into regularized models, you may have noticed that we are only  using the polyfit function to get our overfit models. This is because `SGDRegressor` implements a regularization by default, and there seems to be no way to switch it off. But, you could code a linear regression model algorithm from scratch to see the similar overfitting effect shown by polyfit. 

## Regularization


It seems like overfitting producing large weights. To prevent overfitting, we will add a regularization term in our loss function that penalizes large weights. 

For the **analytical solution**, we use:

$$(X^TX)^{-1} => (X^TX + \lambda I)^{-1}$$ where $\lambda$ is the regularization hyperparameter that controls how much you penalize large weights.

The larger $\lambda$ is, the higher the penalty.


For **gradient descent**:

You need to change the loss function and the weight update.


Here, we will use `SGDRegressor`. We will also change the regularization to either L1 (LASSO) or L2 (Ridge). 

We will be using matrices (and not vectors) for our train and test data in the succeeding cells.

Let's convert our vectors `over_x` and `over_y` to matrices

In [ ]:
X_train = np.expand_dims(over_x, axis=1)
y_train = np.expand_dims(over_y, axis=1)

print('over_x shape', over_x.shape)
print('over_y shape', over_y.shape)
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)

**Sanity Check:** Your results should be the same:
```
over_x shape (20,)
over_y shape (20,)
X_train shape (20, 1)
y_train shape (20, 1)
```
The only difference in the shapes that you should see is the additional dimension.


Let's also reuse `test_x` and `test_y` as our test data.

In [ ]:
X_test = np.expand_dims(test_x, axis=1)
y_test = np.expand_dims(test_y, axis=1)

print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

Let's visualize our `X_train` and `X_test` and their correspoding `y` values.

In [ ]:
plt.scatter(X_train, y_train)
plt.scatter(X_test, y_test, marker='+')

In the following parts, we will NOT use `np.polyfit()` function.

Import the `poly_feature_transform()` function.

In [ ]:
from regularized_linear_regression import poly_feature_transform

Open `regularized_linear_regression.py` file and complete the `poly_feature_transform()` function. This function transforms the features to different polynomial orders.

Implement the `poly_feature_transform()` function. Inline comments should help you in completing the contents of the function.

Transform `over_x` with the degree of 1 and assign the return value to variable `sample_x`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print(sample_x)

__Sanity check:__ Your data should look like this:
```
[[-0.16595599  1.        ]
 [ 0.44064899  1.        ]
 [-0.99977125  1.        ]
 [-0.39533485  1.        ]
 [-0.70648822  1.        ]
 [-0.81532281  1.        ]
 [-0.62747958  1.        ]
 [-0.30887855  1.        ]
 [-0.20646505  1.        ]
 [ 0.07763347  1.        ]
 [-0.16161097  1.        ]
 [ 0.370439    1.        ]
 [-0.5910955   1.        ]
 [ 0.75623487  1.        ]
 [-0.94522481  1.        ]
 [ 0.34093502  1.        ]
 [-0.1653904   1.        ]
 [ 0.11737966  1.        ]
 [-0.71922612  1.        ]
 [-0.60379702  1.        ]]
```

Then, transform `over_x` with the degree of 4 and assign the return value to variable `sample_x`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print(sample_x)

__Sanity check:__ Your data should look like this:
```
[[-1.65955991e-01  2.75413908e-02 -4.57065879e-03  7.58528208e-04
   1.00000000e+00]
 [ 4.40648987e-01  1.94171530e-01  8.55614878e-02  3.77025829e-02
   1.00000000e+00]
 [-9.99771250e-01  9.99542553e-01 -9.99313908e-01  9.99085315e-01
   1.00000000e+00]
 [-3.95334855e-01  1.56289647e-01 -6.17867450e-02  2.44264539e-02
   1.00000000e+00]
 [-7.06488218e-01  4.99125603e-01 -3.52626358e-01  2.49126367e-01
   1.00000000e+00]
 [-8.15322810e-01  6.64751285e-01 -5.41986886e-01  4.41894271e-01
   1.00000000e+00]
 [-6.27479577e-01  3.93730620e-01 -2.47057923e-01  1.55023801e-01
   1.00000000e+00]
 [-3.08878546e-01  9.54059561e-02 -2.94688530e-02  9.10229646e-03
   1.00000000e+00]
 [-2.06465052e-01  4.26278175e-02 -8.80115454e-03  1.81713083e-03
   1.00000000e+00]
 [ 7.76334680e-02  6.02695535e-03  4.67893446e-04  3.63241908e-05
   1.00000000e+00]
 [-1.61610971e-01  2.61181060e-02 -4.22097248e-03  6.82155462e-04
   1.00000000e+00]
 [ 3.70439001e-01  1.37225053e-01  5.08335116e-02  1.88307153e-02
   1.00000000e+00]
 [-5.91095501e-01  3.49393891e-01 -2.06525157e-01  1.22076091e-01
   1.00000000e+00]
 [ 7.56234873e-01  5.71891183e-01  4.32484056e-01  3.27059525e-01
   1.00000000e+00]
 [-9.45224814e-01  8.93449948e-01 -8.44511061e-01  7.98252810e-01
   1.00000000e+00]
 [ 3.40935020e-01  1.16236688e-01  3.96291576e-02  1.35109677e-02
   1.00000000e+00]
 [-1.65390395e-01  2.73539828e-02 -4.52408604e-03  7.48240378e-04
   1.00000000e+00]
 [ 1.17379657e-01  1.37779839e-02  1.61725502e-03  1.89832839e-04
   1.00000000e+00]
 [-7.19226123e-01  5.17286216e-01 -3.72045759e-01  2.67585029e-01
   1.00000000e+00]
 [-6.03797022e-01  3.64570844e-01 -2.20126790e-01  1.32911900e-01
   1.00000000e+00]]
```

Great! Now we can start training our regularized model. Our goal is to achieve good-fit models without overfitting. We will check this by visualizing the hypothesis functions.

Here, we will be using sklearn's `Lasso` and `Ridge` implementations.

____

### Lasso Regularization

Import the `Lasso` class.

In [ ]:
from sklearn.linear_model import Lasso

Instantiate a `Lasso` model with the regularization parameter set to 0.0001 and the number of iterations to 1,000,000. Assign the object to variable `lasso`.

**Note**: The regularization parameter maybe named differently from what was discussed in class, so read the docs.

In [ ]:
# Write your code here


Do not forget that to separately call the function `poly_feature_transform()` since Lasso does not incorporate feature transform unlike the function `np.poly1d()`. For now, set the polynomial order to 1. Assign the return value to variable `poly_x`.

In [ ]:
# Write your code here


Then, train the model.

In [ ]:
# Write your code here


Make predictions on the training data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the train data and assign the return value to variable `train_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('Lasso (order=1) train error: ' + '{:.4f}'.format(train_rmse))

And, finally, we will check if it did well on our test data as well. Make predictions on the test data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the test data and assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('Lasso (order=1) test error: ' + '{:.4f}'.format(test_rmse))

**Question #7:**  What is the test RMSE of our model? Limit to 4 decimal places.

<!--crumb;qna;Q7-->

A: 

Visualize our model. Let's see how well our simple `Lasso` model fared.

In [ ]:
plt.scatter(X_train, y_train)
plt.scatter(X_test,y_test, marker='+')

x_range = np.expand_dims(np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01), axis=1)
y_range = lasso.predict(poly_feature_transform(x_range, poly_order=1))
plt.plot(x_range, y_range, 'k')

plt.title('Lasso (order=1) model')

**Question #8:** What main type of error does the model above have? Bias or variance?

<!--crumb;qna;Q8-->

A: 

Great! We just created a `Lasso` model with a poly degree of 1. Let's now create a `Lasso` model with a higher complexity. 

Instantiate a `Lasso` model with the regularization parameter set to 0.0001 and the number of iterations to 1,000,000. Assign the object to variable `lasso`. Set the polynomial order to 30.

In [ ]:
# Write your code here


Then, train the model.

In [ ]:
# Write your code here


Make predictions on the training data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the train data and assign the return value to variable `train_rmse`.

In [ ]:
# Write your code here


Display the result.

In [ ]:
print('Lasso (order=30) train error: ' + '{:.4f}'.format(train_rmse))

Now, compute the RMSE on the test data and assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print('Lasso (order=30) test error: ' + '{:.4f}'.format(test_rmse))

**Question #9:**  What is the test RMSE of our model? Limit to 4 decimal places.

<!--crumb;qna;Q9-->

A: 

**Question #10:** Recall the train and test results of our `UnregularizedLinearRegressor` with polynomial order 30. 
Describe the train and test error of the `UnregularizedLinearRegressor` model (i.e., is the error high? is the error low?). Describe the train and test error of the `Lasso` model (i.e., is the error high? is the error low?).

<!--crumb;qna;Q10-->

A: 

**Question #11:** What error did the `Lasso` model reduce? Bias or variance error?

<!--crumb;qna;Q11-->

A: 

Visualize our model. Let's see how well our simple `Lasso` model fared.

In [ ]:
plt.scatter(X_train, y_train)
plt.scatter(X_test,y_test, marker='+')

x_range = np.expand_dims(np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01), axis=1)
y_range = lasso.predict(poly_feature_transform(x_range, poly_order=30))
plt.plot(x_range, y_range, 'k')

plt.title('Lasso (order=30) model')

**Question #12:** Compare this with the `Lasso` order=1 model. Which of the following best describes our model: (A) overfit, (B) underfit, or (C) fits the data well?

<!--crumb;qna;Q12-->

A: 

### Visualizing `Lasso` with increasing polynomial orders

From our example above, we can clearly see that our regularized `Lasso` model shows improvement versus the unregularized model. Let's see its behavior across increasing polynomial orders.

In [ ]:
orders = [1, 3, 5, 7, 10, 12, 20, 30, 50]

Instantiate a `Lasso` model with the regularization parameter set to 0.0001 and the number of iterations to 1,000,000. Assign the object to variable `lasso`.

In [ ]:
# Write your code here


To make the following code work, you have to do the following per iteration:
1. Perform polynomial feature transform on the training data according to the current polynomial order
1. Train the model to your transformed train data and the training labels
1. Perform polynomial feature transform `x_range` according to the current polynomial order
1. Make the model run predictions on your transformed `x_range`
1. Collect the weights per model (this is not really needed for the visuals, but we want to see the weights later on)

In [ ]:
weights = {}

plt.figure(figsize=(15, 12))
for i in range(len(orders)):
    order = orders[i]
        
    # Write your code here
    poly_x = None
    
    # Plotting the hypothesis function
    # Plotting the predictions for a range of x (this draws our hypothesis function)
    X_range = np.expand_dims(np.arange(-0.9, 0.9, 0.02), 1)
    
    # Write your code here
    X_range_poly = None
    y_range_predicted = None
    
    # This just plots the i + 1 figure in the 3x3 figure grid
    plt.subplot(3, 3, i + 1)
    
    # Plotting the training data
    plt.scatter(X_train, y_train)
    # Plotting the hypothesis function
    plt.plot(X_range, y_range_predicted, 'b')
    
    # Plotting the optimal answer
    x_range_optimal = np.arange(np.min(X_train), np.max(X_train), 0.01)
    y_range_optimal = np.sin(6 * x_range_optimal)
    plt.plot(x_range_optimal, y_range_optimal, 'k:')
    
    # Change the limit, and order "titles"
    plt.ylim(-2, 2)
    plt.title('Lasso, Order=' + str(order))
    
    # Write your code here
    weights[order] = None

### Checking the Lasso weights
Let's see the effect on the `Lasso` (L1) regularization on the weights

In [ ]:
for i in orders:
    length = weights[i].shape[0]
    bound = np.min([length, 10])
    curweights = weights[i][:bound]
    print('order =', i, end='\t')
    for weight in curweights:
        print(str(weight),end='\t')
    print('')

**Sanity Check:** You should not see the weights straying away from 0 like they did before applying regularization. You will also see some weights turn into 0.0

_______

### Ridge Regularization

Import the `Ridge` class.

In [ ]:
from sklearn.linear_model import Ridge

Instantiate a `Ridge` model with the regularization parameter set to 0.0001 and the solver to `auto`. Assign the object to variable `ridge`.

**Note**: The regularization parameter maybe named differently from what was discussed in class, so read the docs.

In [ ]:
# Write your code here


Do not forget that to separately call the function `poly_feature_transform()` since Ridge does not incorporate feature transform unlike the function `np.poly1d()`. For now, set the polynomial order to 1. Assign the return value to variable `poly_x`.

In [ ]:
# Write your code here
 

Then, train the model.

In [ ]:
# Write your code here


Make predictions on the training data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the train data and assign the return value to variable `train_rmse`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print('Ridge (order=1) train error: ' + '{:.4f}'.format(train_rmse))

And, finally, we will check if it did well on our test data as well. Make predictions on the test data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the test data and assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print('Ridge (order=1) test error: ' + '{:.4f}'.format(test_rmse))

**Question #13:**  What is the test RMSE of our model? Limit to 4 decimal places.

<!--crumb;qna;Q13-->

A: 

Visualize our model. Let's see how well our simple `Ridge` model fared.

In [ ]:
plt.scatter(X_train, y_train)
plt.scatter(X_test,y_test, marker='+')

x_range = np.expand_dims(np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01), axis=1)
y_range = ridge.predict(poly_feature_transform(x_range, poly_order=1))
plt.plot(x_range, y_range, 'k')

plt.title('Ridge (order=1) model')

Just like in `Lasso`, we will also create a `Ridge` model with a higher complexity, where polynomial order is set to 30. 

Instantiate a `Ridge` model with the regularization parameter set to 0.0001 and the solver to `auto`. Assign the object to variable `ridge`. Set the polynomial order to 30.

In [ ]:
# Write your code here


Then, train the model.

In [ ]:
# Write your code here


Make predictions on the training data and assign the return value to variable `y_predicted`.

In [ ]:
# Write your code here


Compute the RMSE on the train data and assign the return value to variable `train_rmse`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print("Ridge (order=30) train error: " + '{:.4f}'.format(train_rmse))

Now, compute the RMSE on the test data and assign the return value to variable `test_rmse`.

In [ ]:
# Write your code here


Display the results.

In [ ]:
print('Ridge (order=30) test error: ' + '{:.4f}'.format(test_rmse))

**Question #14:**  What is the test RMSE of our model? Limit to 4 decimal places.

<!--crumb;qna;Q14-->

A: 

Visualize our model. Let's see how well our `Ridge` (order=30) model performed.

In [ ]:
plt.scatter(X_train, y_train)
plt.scatter(X_test,y_test, marker='+')

x_range = np.expand_dims(np.arange(np.min(over_x), np.max(over_x) + 0.01, 0.01), axis=1)
y_range = ridge.predict(poly_feature_transform(x_range, poly_order=30))
plt.plot(x_range, y_range, 'k')

plt.title('Ridge (order=1) model')

**Question #15:** What error did the `Ridge` model reduce? Bias or variance error?

<!--crumb;qna;Q15-->

A: 

**Question #16:** Between the `UnregularizedLinearRegressor` and the regularized models, which achieved a lower train error? Why does this behavior happen?

<!--crumb;qna;Q16-->

A: 

**Question #17:** Between the `UnregularizedLinearRegressor` and the regularized models, which achieved a lower test error? Why does this behavior happen?

<!--crumb;qna;Q17-->

A: 

### Visualizing `Ridge` with increasing polynomial orders

From our example above, we can clearly see that our regularized `Ridge` model did not overfit the data unlike `UnregularizedLinearRegressor`. Let's also see its behavior across increasing polynomial orders.

In [ ]:
orders = [1, 3, 5, 7, 10, 12, 20, 30, 50]

Instantiate a `Ridge` model with the regularization parameter set to 0.0001 and the solver to `auto`. Assign the object to variable `ridge`.

In [ ]:
# Write your code here


To make the following code work, you have to do the following per iteration:
1. Perform polynomial feature transform on the training data according to the current polynomial order
1. Train the model to your transformed train data and the training labels
1. Perform polynomial feature transform `x_range` according to the current polynomial order
1. Make the model run predictions on your transformed `x_range`
1. Collect the weights per model

In [ ]:
weights = {}

plt.figure(figsize=(15, 12))
for i in range(len(orders)):
    order = orders[i]
        
    # Write your code here
    poly_x = None
    
    # Plotting the hypothesis function
    # Plotting the predictions for a range of x (this draws our hypothesis function)
    X_range = np.expand_dims(np.arange(-0.9, 0.9, 0.02), 1)
    
    # Write your code here
    X_range_poly = None
    y_range_predicted = None
    
    # This just plots the i+1 figure in the 3x3 figure grid
    plt.subplot(3, 3, i + 1)
    
    # Plotting the training data
    plt.scatter(X_train, y_train)
    # Plotting the hypothesis function
    plt.plot(X_range, y_range_predicted, 'b')
    
    # Plotting the optimal answer
    x_range_optimal = np.arange(np.min(X_train), np.max(X_train), 0.01)
    y_range_optimal = np.sin(6 * x_range_optimal)
    plt.plot(x_range_optimal, y_range_optimal, 'k:')
    
    # Change the limit, and order "titles"
    plt.ylim(-2, 2)
    plt.title('Ridge, Order=' + str(order))
    
    # Write your code here
    weights[order] = None

### Checking the Ridge weights
Let's see the effect on the `Ridge` (L2) regularization on the weights

In [ ]:
for i in orders:
    length = weights[i].shape[0]
    bound = np.min([length, 10])
    curweights = weights[i][:bound]
    print('order =', i, end='\t')
    for weight in curweights:
        print(str(weight),end='\t')
    print('')

__Sanity Check:__ You should see the weights being closer to 0 compared to the `UnregularizedLinearRegressor`

____

### Effect of the regularization parameter

Let's try to adjust the regularization parameter and check its effect on the model. 

We'll just reuse our `Ridge` model from the previous cells, but we will change the regularization parameter to `40`.

Instantiate a `Ridge` model with the regularization parameter set to 40 and the solver to `auto`. Assign the object to variable `ridge`.

In [ ]:
# Write your code here


In [ ]:
weights = {}

plt.figure(figsize=(15, 12))
for i in range(len(orders)):
    order = orders[i]
        
    # Write your code here
    poly_x = None
    
    # Plotting the hypothesis function
    # Plotting the predictions for a range of x (this draws our hypothesis function)
    X_range = np.expand_dims(np.arange(-0.9, 0.9, 0.02), 1)
    
    # Write your code here
    X_range_poly = None
    y_range_predicted = None
    
    # This just plots the i+1 figure in the 3x3 figure grid
    plt.subplot(3, 3, i + 1)
    
    # Plotting the training data
    plt.scatter(X_train, y_train)
    # Plotting the hypothesis function
    plt.plot(X_range, y_range_predicted, 'b')
    
    # Plotting the optimal answer
    x_range_optimal = np.arange(np.min(X_train), np.max(X_train), 0.01)
    y_range_optimal = np.sin(6 * x_range_optimal)
    plt.plot(x_range_optimal, y_range_optimal, 'k:')
    
    # Change the limit, and order "titles"
    plt.ylim(-2, 2)
    plt.title('Ridge, Order=' + str(order) + ', $\lambda$ = 40')
    
    # Write your code here
    weights[order] = None

**Question #18:** What happened to the models when we set the regularization parameter to 40 (compared to the original `Ridge` model)? Why are we seeing this behavior?

<!--crumb;qna;Q18-->

A: 

Then, let's do the opposite. 

Instantiate a `Ridge` model with the regularization parameter set to 0 and the solver to `auto`. Assign the object to variable `ridge`.

In [ ]:
# Write your code here


In [ ]:
weights = {}

plt.figure(figsize=(15, 12))
for i in range(len(orders)):
    order = orders[i]
        
    # Write your code here
    poly_x = None
    
    # Plotting the hypothesis function
    # Plotting the predictions for a range of x (this draws our hypothesis function)
    X_range = np.expand_dims(np.arange(-0.9, 0.9, 0.02), 1)
    
    # Write your code here
    X_range_poly = None
    y_range_pred = None
    
    # This just plots the i+1 figure in the 3x3 figure grid
    plt.subplot(3, 3, i + 1)
    
    # Plotting the training data
    plt.scatter(X_train, y_train)
    # Plotting the hypothesis function
    plt.plot(X_range,y_range_pred, 'b')
    
    # Plotting the optimal answer
    x_range_optimal = np.arange(np.min(X_train), np.max(X_train), 0.01)
    y_range_optimal = np.sin(6 * x_range_optimal)
    plt.plot(x_range_optimal, y_range_optimal, 'k:')
    
    # Change the limit, and order "titles"
    plt.ylim(-2, 2)
    plt.title('Ridge, Order=' + str(order) + ', $\lambda$ = 0')
    
    # Write your code here
    weights[order] = None

____

## Another way of preventing overfit models

In class, we discussed that having more data points will prevent overfitting. Let's see what hypothesis function we'll end up with if we have more than 20 data points.

In [ ]:
pts = 1000

# Write your code here
over_x, over_y = None

plt.scatter(over_x,over_y)

We will create an `UnregularizedLinearRegressor` and apply it with different polynomial orders again but now with more training data.

In [ ]:
orders = np.arange(1, 40, 2)

To make the following code work, you have to do the following per iteration:
1. Create an `UnregularizedLinearRegressor` with the polynomial degree matching the current iteration's order
1. Fit the model to our new data
1. Make the model run predictions on our new data

In [ ]:
weights = {}
plt.figure(figsize=(10, 10))
for i in range(len(orders)):
    order = orders[i]

    # Write your code here
    regressor = None
    
    plt.subplot(5, 4, i + 1)
    plt.scatter(over_x, over_y)

    X_range = np.arange(np.min(over_x), np.max(over_x), 0.01)
    
    # Write your code here
    y_range_predicted = None
    
    plt.plot(X_range, y_range_predicted, 'k')
    plt.title('Order = ' + str(orders[i]))

As you can see, even without regularization, the more data we have, our model is more consistent and less likely to overfit. This suggests that overfitting is relative to the **complexity** of the hypothesis function as well as the **number of data points** in your training set.

## Summary

Congratulations!

You have now created a linear regression model that can fit a complex function without running into the problem of overfitting. 

<img src="http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png">
In the figure above, we want to be where the error is the lowest

To **summarize**:
* We definitely do not want to underfit, so we allow feature transform to properly capture data
* But we don't want to overfit either by doing feature transform, so we must make sure that we don't overfit by keeping an eye on the weights.
* We do this by updating our loss function to also consider large weights (weights far from zero) as additional loss
* Weights now update to make sure that training error is minimal and weights be as close to 0 as possible.



Next, we will move on to logistic regression, which builds on top of linear regression. Unlike linear regression, logistic regression is a classification technique.

See you in the next notebook!

### <center>fin</center>


<!-- DO NOT MODIFY OR DELETE THIS -->

<sup>made/compiled by daniel stanley tan & courtney anne ngo 🐰 & thomas james tiam-lee</sup> <br>
<sup>for comments, corrections, suggestions, please email:</sup>
<sup> danieltan07@gmail.com & courtneyngo@gmail.com & thomasjamestiamlee@gmail.com</sup><br>
<sup>please cc your instructor, too</sup>
<!-- DO NOT MODIFY OR DELETE THIS -->